In [1]:
from lusidtools.jupyter_tools import toggle_code
from IPython.core.display import HTML

""" Creating portfolios with different tax lot management methods

This notebook demonstrates how to create transaction portfolios under different tax lot accounting methodologies.

Attributes
----------
transactions
holdings
taxlots
accounting
"""

toggle_code("Toggle Docstring")

# Tax Lot Management

Contents: 

* [1. Initial Setup](#1.-Initial-Setup)
* [2. Loading our Data](#2.-Loading-our-Data)
    * [2.1 Create Portfolio](#2.1-Create-Portfolio)
    * [2.2 Create Instruments](#2.2-Create-Instruments)
    * [2.3 Upload Transactions](#2.3-Upload-Transactions)
* [3. Cost Basis Comparison](#3.-Cost-Basis-Comparison)
    * [3.1 Average Cost](#3.1-Average-Cost)
    * [3.2 First In First Out](#3.2-First-In-First-Out-(FIFO)) 
    * [3.3 Last In Last Out](#3.3-Last-In-Last-Out-(LIFO))
    * [3.4 Highest First](#3.4-Highest-First) 
    * [3.5 Lowest First](#3.5-Lowest-First)
* [4. Corporate Actions](#4.-Corporate-Actions)
* [5. Demo Walkthrough](#5.-Demo-Walkthrough)


Portfolios in LUSID can be created using any of the following tax lot accounting methods:
- Average Cost (default in LUSID)
- First In First Out (FIFO)
- Last In First Out (LIFO)
- Lowest First
- Highest First

These will determine how tax lots are used to update the cost basis of a transaction portfolio when booking various transactions.
Helpful KB articles:
- [What are the supported tax-lot accounting methods in LUSID?](https://support.lusid.com/knowledgebase/article/KA-01886/en-us)
- [How do I handle different tax lot accounting conventions?](https://support.lusid.com/knowledgebase/article/KA-01887/en-us)

## 1. Initial Setup<a name="initialsetup"></a>
This section will set up the parameters and methods used in section 2, to compare accounting methods.

In [2]:
# Import LUSID
import lusid
import lusid.models as models
import pandas as pd
from IPython.core.display import HTML

# Import Libraries
import pytz
from lusidjam import RefreshingToken
import json
import os
from datetime import datetime
from dateutil.parser import parse
from enum import Enum
from datetime import timedelta

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

pd.set_option('display.float_format', lambda x: f'{x:,.1f}')

api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename = secrets_path,
    app_name="LusidJupyterNotebook")

print('LUSID Environment Initialised')
print('LUSID version : ', api_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().build_version)

LUSID Environment Initialised
LUSID version :  0.6.11194.0


In [3]:
# Create necessary API factories
transaction_portfolios_api = api_factory.build(lusid.api.TransactionPortfoliosApi)
derived_portfolios_api = api_factory.build(lusid.api.DerivedTransactionPortfoliosApi)
property_definitions_api = api_factory.build(lusid.api.PropertyDefinitionsApi)
instruments_api = api_factory.build(lusid.api.InstrumentsApi)
corporate_action_sources_api = api_factory.build(lusid.api.CorporateActionSourcesApi)

# 2. Loading our Data<a name = "loading-our-data"></a>

In [4]:
# Specify scope and load data
txns = pd.read_csv("data/taxlot_accounting_transactions.csv",parse_dates=['transaction_date','settlement_date'], date_parser = lambda x : parse(x).strftime('%Y-%m-%d'))
scope = "taxlot_management_examplenb"
txns.head()

,transaction_id,type,units,transaction_price,transaction_date,settlement_date,total_consideration,client_internal,ticker,name,sleeve
0,txn_000,FundsIn,"80,000,000",1.0,2020-03-04,2020-03-05,"80,000,000",CASH_USD,USD,Cash USD,Sleeve 1
1,txn_001,FundsIn,"95,000,000",1.0,2020-03-04,2020-03-05,"95,000,000",CASH_USD,USD,Cash USD,Sleeve 2
2,txn_002,Buy,"100,000",167.6,2020-03-05,2020-03-08,"16,763,000",EQUITY_APPL,AAPL,Apple,Sleeve 1
3,txn_003,Buy,"100,000",163.0,2020-03-06,2020-03-09,"16,298,000",EQUITY_APPL,AAPL,Apple,Sleeve 1
4,txn_004,Buy,"100,000",165.4,2020-03-07,2020-03-10,"16,542,000",EQUITY_APPL,AAPL,Apple,Sleeve 1


## 2.1 Create Portfolio<a name = "create-portfolio"></a>
When creating a transaction portfolio in LUSID, we pass in a parameter of the `accounting_method` to determine how the tax lots are calculated. If not specified, **Average Cost** is the default method used in LUSID.

In [5]:
# Create corporate action source, this is covered further in section 4
ca_code = "tax_lot_stock_split"
try:

    source_request = models.CreateCorporateActionSourceRequest(
        scope=scope,
        code=ca_code,
        display_name="Tax Lot Corporate Action Source",
        description="Corporate Actions source for tax lot sample notebook",
    )

    corporate_action_sources_api.create_corporate_action_source(
        create_corporate_action_source_request=source_request
    )

except lusid.ApiException as e:
    print(json.loads(e.body)["title"])

Could not create a Corporate Action Source with id tax_lot_stock_split because it already exists in scope taxlot_management_examplenb.


In [6]:
# Create sleeve property for sub-holding key
try:
    resp = property_definitions_api.create_property_definition(
        create_property_definition_request=models.CreatePropertyDefinitionRequest(
            domain="Transaction",
            scope=scope,
            code="sleeve",
            value_required=None,
            display_name="Sleeve",
            data_type_id=models.ResourceId(scope="system", code="string"),
            life_time=None,
        )
    )
    print(f"{resp.key} property created")
    
except lusid.ApiException as e:
    if json.loads(e.body)["code"] == 124: # PropertyAlreadyExists
        print(json.loads(e.body)["title"])
    else:
        raise e

Error creating Property Definition 'Transaction/taxlot_management_examplenb/sleeve' because it already exists.


In [7]:
def create_upload_portfolio(name, accounting_method):
    try:
        created_date = "2010-01-01T00:00:00.000000+00:00"

        # Create request body
        portfolio_request = models.CreateTransactionPortfolioRequest(
            display_name=f"Tax Lot Management Example - {accounting_method}",
            code=name,
            base_currency="USD",
            corporate_action_source_id=lusid.ResourceId(scope=scope, code=ca_code),
            accounting_method=accounting_method, # If not specified - AverageCost is the default accounting method.
            created=created_date,
            sub_holding_keys=[f"Transaction/{scope}/sleeve"],
        )

        # Upload new portfolio to LUSID
        response = transaction_portfolios_api.create_portfolio(
            scope=scope, create_transaction_portfolio_request=portfolio_request
        )

        created = response.version.effective_from
        print(
        f"Portfolio '{response.id.code}', in scope {scope} created effective from: "
        f"{created.year}/"
        f"{created.month}/"
        f"{created.day}")
    except lusid.ApiException as e:
        print(json.loads(e.body)["title"])

In [8]:
def create_derived_portfolio(portfolio_code, parent_code, accounting_method, ca_code):
    try:
        derived_portfolio_request = models.CreateDerivedTransactionPortfolioRequest(
            display_name=f"Tax Lot Management Example - {accounting_method}",
            code=portfolio_code,
            parent_portfolio_id=models.ResourceId(scope=scope, code= parent_code),
            created="2020-01-01",
            corporate_action_source_id=lusid.ResourceId(scope=scope, code=ca_code),
            accounting_method=accounting_method,
            sub_holding_keys=[f"Transaction/{scope}/sleeve"],
        )
        response = derived_portfolios_api.create_derived_portfolio(
            scope=scope,
            create_derived_transaction_portfolio_request = derived_portfolio_request)
        created = response.version.effective_from
        print(
        f"Derived portfolio '{response.id.code}', in scope {scope} created effective from: "
        f"{created.year}/"
        f"{created.month}/"
        f"{created.day}")

    except lusid.ApiException as e:
        print(json.loads(e.body)["title"])

In [9]:
# Create a parent portfolio.
portfolio_name_avg = "tax-lot-avgcost"
create_upload_portfolio(portfolio_name_avg, "AverageCost")

Portfolio 'tax-lot-avgcost', in scope taxlot_management_examplenb created effective from: 2010/1/1


In [10]:
# Create portfolios derived from the AverageCost portfolio. (Tax lot type of the parent portfolio does not affect the derived
# portfolios.)
portfolio_name_fifo = "tax-lot-fifo"
create_derived_portfolio(portfolio_name_fifo, portfolio_name_avg, "FirstInFirstOut", ca_code)

portfolio_name_lifo = "tax-lot-lifo"
create_derived_portfolio(portfolio_name_lifo, portfolio_name_avg, "LastInFirstOut", ca_code)

portfolio_name_high = "tax-lot-highestcost"
create_derived_portfolio(portfolio_name_high, portfolio_name_avg, "HighestCostFirst", ca_code)

portfolio_name_low = "tax-lot-lowestcost"
create_derived_portfolio(portfolio_name_low, portfolio_name_avg, "LowestCostFirst", ca_code)

Derived portfolio 'tax-lot-fifo', in scope taxlot_management_examplenb created effective from: 2010/1/1
Derived portfolio 'tax-lot-lifo', in scope taxlot_management_examplenb created effective from: 2010/1/1
Derived portfolio 'tax-lot-highestcost', in scope taxlot_management_examplenb created effective from: 2010/1/1
Derived portfolio 'tax-lot-lowestcost', in scope taxlot_management_examplenb created effective from: 2010/1/1


## 2.2 Create Instruments<a name = "create-instruments"></a>

In [11]:
# Create instruments
batch_upsert_request = {}
uploaded = []

for index, row in txns.iterrows():
    client_internal = row['client_internal']
    ticker = row['ticker']
    name = row['name']
    
    if client_internal.startswith("CASH") or client_internal in uploaded:
        continue
    else:
        batch_upsert_request[client_internal] = models.InstrumentDefinition(
            name=name,
            identifiers={ 
                "ClientInternal": models.InstrumentIdValue(value=client_internal),
                "Ticker": models.InstrumentIdValue(value=ticker)
            },
            definition=models.Equity(
                instrument_type="Equity",
                dom_ccy="USD"
            )
        )
        uploaded.append(client_internal)
        

# Upsert new instruments to LUSID
instrument_response = instruments_api.upsert_instruments(
    request_body=batch_upsert_request
)

# Check response was successful
if len(instrument_response.failed) > 0:
    raise AssertionError("Instruments upsert failed. Inspect response for more detail")
    
print(f"{len(instrument_response.values)} instruments uploaded")

6 instruments uploaded


## 2.3 Upload Transactions<a name = "upload-transactions"></a>

In [12]:
def upload_transactions_to_portfolio(portfolio_name, transactions):
    # Upsert transactions
    transactions_request = []
    txn_response = []

    for index, row in transactions.iterrows():

        if row["client_internal"].startswith("CASH"):
            instrument_identifier = {"Instrument/default/Currency": "USD"}

        else:
            instrument_identifier = {
                    "Instrument/default/ClientInternal": row["client_internal"]
                }

        # Build request body
        transactions_request.append(
            models.TransactionRequest(
                transaction_id=row["transaction_id"],
                type=row["type"],
                instrument_identifiers=instrument_identifier,
                transaction_date=row["transaction_date"].strftime("%Y-%m-%d"),
                settlement_date=row["settlement_date"].strftime("%Y-%m-%d"),
                units=row["units"],
                transaction_price=models.TransactionPrice(price=row["transaction_price"], type="Price"),
                total_consideration=models.CurrencyAndAmount(
                    amount=row["total_consideration"], currency="USD"
                ),
                properties={
                    f"Transaction/{scope}/sleeve": 
                    models.PerpetualProperty(
                        key=f"Transaction/{scope}/sleeve",
                        value=models.PropertyValue(label_value=row['sleeve'])
                    )
                }
            )
        )

        # Make upsert transactions call to LUSID
        txn_response.append(
            transaction_portfolios_api.upsert_transactions(
                scope=scope, code=portfolio_name, transaction_request=transactions_request
            )
        )

    print(f"{len(txn_response)} transactions upserted to portfolio: {portfolio_name}")

In [13]:
# Upload transactions to parent portfolio. The transactions will be automatically added to the derived portfolios.
upload_transactions_to_portfolio(portfolio_name_avg, txns)

28 transactions upserted to portfolio: tax-lot-avgcost


In [14]:
# Show transactions in chronological order
txns.sort_values('transaction_date',ascending=True).head()

,transaction_id,type,units,transaction_price,transaction_date,settlement_date,total_consideration,client_internal,ticker,name,sleeve
0,txn_000,FundsIn,"80,000,000",1.0,2020-03-04,2020-03-05,"80,000,000",CASH_USD,USD,Cash USD,Sleeve 1
1,txn_001,FundsIn,"95,000,000",1.0,2020-03-04,2020-03-05,"95,000,000",CASH_USD,USD,Cash USD,Sleeve 2
6,txn_006,Buy,"70,000",288.4,2020-03-04,2020-03-08,"20,191,500",EQUITY_MSFT,MSFT,Microsoft,Sleeve 2
18,txn_018,Buy,"25,000",162.5,2020-03-05,2020-03-08,"4,063,250",EQUITY_JNJ,JNJ,Johnson & Johnson,Sleeve 1
14,txn_014,Buy,"18,000",279.3,2020-03-05,2020-03-08,"5,027,580",EQUITY_NVDA,NVDA,NVIDIA,Sleeve 2


# 3. Cost Basis Comparison<a name = "cost-basis-comp"></a>
Holdings before our sell transaction show the three distinct tax lots and corresponding transactions. We can see that each tax lot has a distinct cost basis based on the transaction price and total consideration of each transaction.

When calling `get_holdings()` in the `display_holding_positions_by_taxlot` method, setting the flag `by_taxlots=true` returns the holdings separated by tax lots, as shown below.

In [15]:
# Prints quick summary from a get_holdings() response of positions for a given effective_at date broken down by tax lots
def display_holding_positions_by_taxlot(effective_at, portfolio_name, show_taxlots = True):
    # Get holdings
    response = transaction_portfolios_api.get_holdings(
    scope=scope,
    code=portfolio_name,
    effective_at = effective_at.isoformat(),
    property_keys=["Instrument/default/Name"],
    by_taxlots=show_taxlots
    )

    # Inspect holdings response for the given effective_at day
    hld = [i for i in response.values]

    names = []
    cost = []
    units = []
    txnid = []
    pchprice = []
    pchdate = []

    for item in hld:
        if item.holding_type_name == "Position":
            names.append(item.properties["Instrument/default/Name"].value.label_value)
            cost.append(item.cost.amount)
            units.append(item.units)
            if show_taxlots:
                txnid.append(item.properties["Holding/default/TaxlotId"].value.label_value)
                pchprice.append(item.properties["Holding/default/TaxlotPurchasePrice"].value.metric_value.value)
                pchdate.append(item.properties["Holding/default/TaxlotPurchaseDate"].value.label_value.replace("T00:00:00.0000000+00:00",""))
                
    data = {"cost_basis": cost, "units": units}
    if show_taxlots:
        data = {"transaction_id": txnid, **data , "purchase_price":pchprice, "purchase_date":pchdate}
    return pd.DataFrame(data=data, index=names)

In [16]:
# Displays a link to view portfolio in LUSID
def print_url_to_holdings(effective_at, portfolio_name):
    date = effective_at.strftime('%Y-%m-%d')
    api_url = api_factory.api_client.configuration._base_path.replace("api","")
    display(HTML(f'<a href="{api_url}app/dashboard/holdings?scope=taxlot_management_examplenb&code={portfolio_name}&entityType=Portfolio&taxLots=true&effectiveDate={date}" target="_blank">See holdings positions by tax lot in LUSID</a>'))

In [17]:
display_holding_positions_by_taxlot(datetime(year=2020, month=3, day=7, tzinfo=pytz.UTC), portfolio_name_fifo)

,transaction_id,cost_basis,units,purchase_price,purchase_date
Microsoft,txn_006,"20,191,500.0","140,000.0",288.4,2020-03-04
Microsoft,txn_007,"19,903,800.0","140,000.0",284.3,2020-03-05
Microsoft,txn_008,"9,816,100.0","70,000.0",140.2,2020-03-07
Apple,txn_002,"16,763,000.0","100,000.0",167.6,2020-03-05
Apple,txn_003,"16,298,000.0","100,000.0",163.0,2020-03-06
Apple,txn_004,"16,542,000.0","100,000.0",165.4,2020-03-07
Berkshire Hathaway,txn_010,"3,230,900.0","10,000.0",323.1,2020-03-05
Berkshire Hathaway,txn_011,"3,202,000.0","10,000.0",320.2,2020-03-06
Berkshire Hathaway,txn_012,"3,256,800.0","10,000.0",325.7,2020-03-07
NVIDIA,txn_014,"5,027,580.0","18,000.0",279.3,2020-03-05


In [18]:
print_url_to_holdings(datetime(year=2020, month=3, day=7, tzinfo=pytz.UTC), portfolio_name_fifo)

We'll now look at the final positions of our portfolios under the 5 accounting methods described above. The units sold for each quity can be seen in the 'units' column.

## 3.1 Average Cost
Average Cost uses the average price of the final holdings of our portfolios. In the case of **Apple**, for example, this is $ \frac{49,603,000}{300,000} = 165.34.$
When using the **Average Cost** method, our cost basis is averaged across all transactions. Splitting out our holdings by tax lot is thus not applicable. The total cost basis after the sale will be calculated as $165.34 \times 150,000 = 24,801,500$.
**Average Cost** is the default accounting method for a portfolio created in LUSID.

In [19]:
display_holding_positions_by_taxlot(datetime(year=2020, month=3, day=14, tzinfo=pytz.UTC), portfolio_name_avg, show_taxlots=False)

,cost_basis,units
Microsoft,"38,503,080.0","270,000.0"
Apple,"24,801,500.0","150,000.0"
Berkshire Hathaway,"4,844,850.0","15,000.0"
NVIDIA,"556,443.8","2,000.0"
Johnson & Johnson,"4,074,375.0","25,000.0"
Johnson & Johnson,"4,074,375.0","25,000.0"
Pepsico,"1,847,200.0","10,000.0"


In [20]:
print_url_to_holdings(datetime(year=2020, month=3, day=7, tzinfo=pytz.UTC), portfolio_name_avg)

## 3.2 First In First Out (FIFO)
For **Apple**, using FIFO, our first tax lot (all 100,000 units coming from `txn_002`) will be fully sold, while half of our second tax lot (50,000 units coming from `txn_003`) will be sold. Below we show the remaining holdings with 50,000 units bought in `txn_003` and all the units bought in `txn_004` in separate tax lots.

In [21]:
display_holding_positions_by_taxlot(datetime(year=2020, month=3, day=14, tzinfo=pytz.UTC), portfolio_name_fifo)

,transaction_id,cost_basis,units,purchase_price,purchase_date
Microsoft,txn_006,"8,653,500.0","60,000.0",288.4,2020-03-04
Microsoft,txn_007,"19,903,800.0","140,000.0",284.3,2020-03-05
Microsoft,txn_008,"9,816,100.0","70,000.0",140.2,2020-03-07
Apple,txn_003,"8,149,000.0","50,000.0",163.0,2020-03-06
Apple,txn_004,"16,542,000.0","100,000.0",165.4,2020-03-07
Berkshire Hathaway,txn_011,"1,601,000.0","5,000.0",320.2,2020-03-06
Berkshire Hathaway,txn_012,"3,256,800.0","10,000.0",325.7,2020-03-07
NVIDIA,txn_016,"551,760.0","2,000.0",275.9,2020-03-07
Johnson & Johnson,txn_019,"4,085,500.0","25,000.0",163.4,2020-03-06
Johnson & Johnson,txn_021,"4,085,500.0","25,000.0",163.4,2020-03-06


In [22]:
print_url_to_holdings(datetime(year=2020, month=3, day=7, tzinfo=pytz.UTC), portfolio_name_fifo)

## 3.3 Last In Last Out (LIFO)
For **Apple**, using LIFO, our first tax lot (all 100,000 units coming from `txn_004`) will be fully sold, while half of our second tax lot (50,000 units coming from `txn_003`) will be sold.
The difference in the total cost basis compared to the **FIFO** method, is due to the difference in total consideration between `txn_002` and `txn_004`.

In [23]:
display_holding_positions_by_taxlot(datetime(year=2020, month=3, day=14, tzinfo=pytz.UTC), portfolio_name_lifo)

,transaction_id,cost_basis,units,purchase_price,purchase_date
Microsoft,txn_006,"20,191,500.0","140,000.0",288.4,2020-03-04
Microsoft,txn_007,"18,482,100.0","130,000.0",284.3,2020-03-05
Apple,txn_002,"16,763,000.0","100,000.0",167.6,2020-03-05
Apple,txn_003,"8,149,000.0","50,000.0",163.0,2020-03-06
Berkshire Hathaway,txn_010,"3,230,900.0","10,000.0",323.1,2020-03-05
Berkshire Hathaway,txn_011,"1,601,000.0","5,000.0",320.2,2020-03-06
NVIDIA,txn_014,"558,620.0","2,000.0",279.3,2020-03-05
Johnson & Johnson,txn_018,"4,063,250.0","25,000.0",162.5,2020-03-05
Johnson & Johnson,txn_020,"4,063,250.0","25,000.0",162.5,2020-03-05
Pepsico,txn_024,"1,847,200.0","10,000.0",184.7,2020-03-05


In [24]:
print_url_to_holdings(datetime(year=2020, month=3, day=7, tzinfo=pytz.UTC), portfolio_name_lifo)

## 3.4 Highest First
Using Highest First, the equities with the highest transaction price will be sold first. Take **Apple** for example, in the sell transaction of 150,000 units, all units from `txn_002` and 50,000 units of `txn_004` will be sold. All units from `txn_003` and 50,000 units from `txn_004` remain in the cost basis as shown in the display below.

In [25]:
display_holding_positions_by_taxlot(datetime(year=2020, month=3, day=14, tzinfo=pytz.UTC), portfolio_name_high)

,transaction_id,cost_basis,units,purchase_price,purchase_date
Microsoft,txn_006,"8,653,500.0","60,000.0",288.4,2020-03-04
Microsoft,txn_007,"19,903,800.0","140,000.0",284.3,2020-03-05
Microsoft,txn_008,"9,816,100.0","70,000.0",140.2,2020-03-07
Apple,txn_003,"16,298,000.0","100,000.0",163.0,2020-03-06
Apple,txn_004,"8,271,000.0","50,000.0",165.4,2020-03-07
Berkshire Hathaway,txn_010,"1,615,450.0","5,000.0",323.1,2020-03-05
Berkshire Hathaway,txn_011,"3,202,000.0","10,000.0",320.2,2020-03-06
NVIDIA,txn_016,"551,760.0","2,000.0",275.9,2020-03-07
Johnson & Johnson,txn_018,"4,063,250.0","25,000.0",162.5,2020-03-05
Johnson & Johnson,txn_020,"4,063,250.0","25,000.0",162.5,2020-03-05


In [26]:
print_url_to_holdings(datetime(year=2020, month=3, day=7, tzinfo=pytz.UTC), portfolio_name_high)

## 3.5 Lowest First
Using Lowest First, the equities with the lowest transaction price will be sold first. For **Apple**, in the sell transaction of 150,000 units, all units from `txn_003` and 50,000 units of `txn_004`. All units from `txn_002` and 50,000 units from `txn_004` remain in the cost basis as shown in the display below.
The difference in the cost basis compared to the **Highest First** method, is due to the difference in total consideration between `txn_002` and `txn_004`.

In [27]:
display_holding_positions_by_taxlot(datetime(year=2020, month=3, day=14, tzinfo=pytz.UTC), portfolio_name_low)

,transaction_id,cost_basis,units,purchase_price,purchase_date
Microsoft,txn_006,"20,191,500.0","140,000.0",288.4,2020-03-04
Microsoft,txn_007,"18,482,100.0","130,000.0",284.3,2020-03-05
Apple,txn_002,"16,763,000.0","100,000.0",167.6,2020-03-05
Apple,txn_004,"8,271,000.0","50,000.0",165.4,2020-03-07
Berkshire Hathaway,txn_010,"1,615,450.0","5,000.0",323.1,2020-03-05
Berkshire Hathaway,txn_012,"3,256,800.0","10,000.0",325.7,2020-03-07
NVIDIA,txn_014,"558,620.0","2,000.0",279.3,2020-03-05
Johnson & Johnson,txn_019,"4,085,500.0","25,000.0",163.4,2020-03-06
Johnson & Johnson,txn_021,"4,085,500.0","25,000.0",163.4,2020-03-06
Pepsico,txn_024,"1,847,200.0","10,000.0",184.7,2020-03-05


In [28]:
print_url_to_holdings(datetime(year=2020, month=3, day=7, tzinfo=pytz.UTC), portfolio_name_low)

## 4. Corporate Actions

Below, we apply a 2 for 1 stock split corporate action on Microsoft shares. This has the effect of doubling the units of an equity that we hold.

In [29]:
# Set conditions of Coroporate Action
transitions = [
    models.CorporateActionTransition(
        input_transition=models.CorporateActionTransitionComponentRequest(
            instrument_identifiers={
                "Instrument/default/ClientInternal": "EQUITY_MSFT"
            },
            units_factor=1,
            cost_factor=1,
        ),
        output_transitions=[
            models.CorporateActionTransitionComponentRequest(
                instrument_identifiers={
                    "Instrument/default/ClientInternal": "EQUITY_MSFT"
                },
                units_factor=2,
                cost_factor=1,
            )
        ],
    )
]

# Creat Coporate Action
split_request = models.UpsertCorporateActionRequest(
    corporate_action_code="SS001",
    announcement_date=datetime(year=2020, month=3, day=5, tzinfo=pytz.UTC),
    ex_date=datetime(year=2020, month=3, day=6, tzinfo=pytz.UTC),
    record_date=datetime(year=2020, month=3, day=6, tzinfo=pytz.UTC),
    payment_date=datetime(year=2020, month=3, day=6, tzinfo=pytz.UTC),
    transitions=transitions,
)

result = corporate_action_sources_api.batch_upsert_corporate_actions(
    scope=scope, code=ca_code, upsert_corporate_action_request=[split_request]
)

Before the stock split.

In [30]:
display_holding_positions_by_taxlot(datetime(year=2020, month=3, day=5, tzinfo=pytz.UTC), portfolio_name_low).query("index == 'Microsoft'")

,transaction_id,cost_basis,units,purchase_price,purchase_date
Microsoft,txn_006,"20,191,500.0","70,000.0",288.4,2020-03-04
Microsoft,txn_007,"19,903,800.0","70,000.0",284.3,2020-03-05


After the stock split. Note how the units have doubled for Microsoft.

In [31]:
display_holding_positions_by_taxlot(datetime(year=2020, month=3, day=7, tzinfo=pytz.UTC), portfolio_name_low).query("index == 'Microsoft'")

,transaction_id,cost_basis,units,purchase_price,purchase_date
Microsoft,txn_006,"20,191,500.0","140,000.0",288.4,2020-03-04
Microsoft,txn_007,"19,903,800.0","140,000.0",284.3,2020-03-05
Microsoft,txn_008,"9,816,100.0","70,000.0",140.2,2020-03-07


## 5. Demo Walkthrough

This sections details some useful links and data that helps to illustrate the taxlot offering we can provide.

1. [Initialise Functions](#5.1-Initialise-Functions)
2. [Show Holdings](#5.2-Show-Holdings)
3. [Show Transactions](#5.3-Show-Transactions)
4. [Corporate Actions](#5.4-Corporate-Actions)
5. [Gain and Loss](#5.5-Gain-and-Loss)

Key Dates:
- **04/03/2020** : Cash paid in and first transaction for MSFT only
- **05/03/2020** : Cash settles and majority of equities get first transaction
- **06/03/2020** : Corporate action stock split (2 for 1) for Microsoft
- **07/03/2020** : Final Buy transactions executed
- **08/03/2020** : All sell transactions executed
- **10/03/2020** : All Buy transactions settled
- **11/03/2020** : All Sell transactions settled

## 5.1 Initialise Functions

Declare the functions that we will use throughout this demo. We have done this at the top of this section to help with readibility.

In [32]:
# Demo class for below function
class Demo(Enum):
    HOLDINGS = "holdings"
    TRANSACTIONS = "transactions"
    CORPORATE_ACTIONS = "actions"

# Generate links for each demo type
def generate_href(demo_type, effective_at, portfolio_list=None):
    # Get users domain from API Factory
    api_url = api_factory.api_client.configuration._base_path.replace("api","")
    # Create URLs
    if demo_type.name != "HOLDINGS":
        from_date = (effective_at - timedelta(60)).strftime('%Y-%m-%d')
        to_date = effective_at.strftime('%Y-%m-%d')
        # Create link for Corporate Actions
        if demo_type.name == "CORPORATE_ACTIONS":
            display(HTML(f'''<b>Corporate Actions:</b> <a href="{api_url}app/data-management/actions?actionsDateFrom={from_date}&actionsDateTo={to_date}&corporateSourceScope=taxlot_management_examplenb&corporateSourceCode=tax_lot_stock_split&corporateSourceName=Tax Lot Corporate Action Source" target="_blank">See corporate actions in LUSID</a>'''))
            return
        # Format date for Transactions
        date_string = f"dateFrom={from_date}&dateTo={to_date}"
    else:
        # Format date for Holdings
        date_string = f"effectiveDate={effective_at.strftime('%Y-%m-%d')}"
        
    # Create links for Holdings and Transactions        
    if portfolio_list:
        for portfolio_name in portfolio_list:
            display(HTML(f'''<b>{portfolio_name[0]}:</b> <a href="{api_url}app/dashboard/{demo_type.value}?scope=taxlot_management_examplenb&code={portfolio_name[1]}&entityType=Portfolio&taxLots=true&{date_string}" target="_blank">See {demo_type.value} by tax lot in LUSID</a>'''))
    else:
        raise Exception(f"Parameter 'portfolio_list' cannot be None for {demo_type}")

# For taxlot comparison
def get_taxlot_facts(taxlot_dfs, taxlot_names):
    gnl = []
    prices = []
    sell_prices = []
    # Get data from each tax lot 
    for df in taxlot_dfs:
        row = df.loc[df['type'] == "Sell"]
        prices.append(row['price'].item())
        sell_prices.append(row['sell_price'].item())
        gnl.append(row['gain/loss'].item())
    # Output the HTML
    display(
        HTML(
            f"""
            <p><b>Highest Gain Tax Lot</b> : {taxlot_names[gnl.index(max(gnl))]}</p>
            <p><b>Gain/Loss</b> : {max(gnl)}</p>
            <p><b>Market Price</b> : {prices[gnl.index(max(gnl))]}</p>
            <p><b>Sell Price</b> : {sell_prices[gnl.index(max(gnl))]}</p>
            <br>
            <p><b>Greatest Loss Tax Lot</b> : {taxlot_names[gnl.index(min(gnl))]}</p>
            <p><b>Gain/Loss</b> : {min(gnl)}</p>
            <p><b>Market Price</b> : {prices[gnl.index(min(gnl))]}</p>
            <p><b>Sell Price</b> : {sell_prices[gnl.index(min(gnl))]}</p>
            """
        )
    )

# Get the gain and loss data
def get_gl(val, pricing=False):
    if not pricing:
        return val.realised_gain_loss[0]
    else:
        sell_units = val.realised_gain_loss[0].units
        sell_amount = val.realised_gain_loss[0].cost_portfolio_ccy.amount
        return sell_amount/sell_units

# Get name of instrument
def get_name(val):
    if 'Instrument/default/ClientInternal' in val.instrument_identifiers:
        return val.instrument_identifiers['Instrument/default/ClientInternal']
    elif 'Instrument/default/Currency' in val.instrument_identifiers:
        return val.instrument_identifiers['Instrument/default/Currency']

# Get gain & loss data by building transactions
def build_transactions(effective_at, portfolio_name):
    from_date = (effective_at - timedelta(60)).strftime('%Y-%m-%d')
    to_date = effective_at.strftime('%Y-%m-%d')
    sleeve_key = 'Transaction/taxlot_management_examplenb/sleeve'

    result = transaction_portfolios_api.build_transactions(
        scope=scope,
        code=portfolio_name,
        transaction_query_parameters=models.TransactionQueryParameters(
            start_date=from_date,
            end_date=to_date
        )
    )

    vals = result.values # for readibility
    gain_loss_data = {
        "txn_date": [val.transaction_date.date() for val in vals],
        "name": [get_name(val) for val in vals],
        "luid": [val.instrument_uid for val in vals],
        "sleeve": [val.properties[sleeve_key].value.label_value for val in vals],
        "type": [val.type for val in vals],
        "price": [val.transaction_price.price for val in vals],
        "units": [val.units for val in result.values],
        "amount": [val.transaction_amount for val in vals],
        "sell_units": [get_gl(val).units if val.realised_gain_loss else None for val in vals],
        "sell_price": [get_gl(val, True) if val.realised_gain_loss else None for val in vals],
        "sell_amount": [get_gl(val).cost_portfolio_ccy.amount if val.realised_gain_loss else None for val in vals],
        "gain/loss": [get_gl(val).realised_market.amount if val.realised_gain_loss else None for val in vals],
    }    

    return pd.DataFrame(data=gain_loss_data).sort_values(by=['luid', 'txn_date'])

In [33]:
portfolio_list = [
    ["Average Cost", portfolio_name_avg],
    ["FIFO", portfolio_name_fifo],
    ["LIFO", portfolio_name_lifo],
    ["Highest First", portfolio_name_high],
    ["Lowest First", portfolio_name_low]
]

## 5.2 Show Holdings

Holdings links (with taxlots on). Try grouping by instrument and then by sleeve.

In [34]:
generate_href(Demo.HOLDINGS, datetime(year=2020, month=3, day=11, tzinfo=pytz.UTC), portfolio_list)

## 5.3 Show Transactions

Generate links for transaction screens. We recommend filtering for a single equity (eg. Pepsico) and showing how its transactions vary depending on the tax lot.

In [35]:
generate_href(Demo.TRANSACTIONS, datetime(year=2020, month=3, day=11, tzinfo=pytz.UTC), portfolio_list)                                

## 5.4 Corporate Actions


First, view the corporate actions in the corporate actions screen. You should see the Microsoft equity being impacted.

In [36]:
generate_href(Demo.CORPORATE_ACTIONS, datetime(year=2020, month=3, day=11, tzinfo=pytz.UTC)) 

Next, view the corporate actions within the transaction screen. To find them, filter your transactions for Microsoft and look for an "Adjustment Increase" transaction type. See how the values vary between the average tax lot and the rest.

In [37]:
generate_href(Demo.TRANSACTIONS, datetime(year=2020, month=3, day=11, tzinfo=pytz.UTC), portfolio_list)                                

## 5.5 Gain and Loss

Below, we get the gain and loss of each strategy for a single equity, and present it in a DataFrame. You can adjust which equity is being compared by modifying the `conditions` variable

In [38]:
# change condition to view other equities eg. EQUITY_MSFT etc.
conditions = 'name == "EQUITY_MSFT"'

effective_at = datetime(year=2020, month=3, day=14, tzinfo=pytz.UTC)

avg_df = build_transactions(effective_at, portfolio_name_avg).query(conditions)
fifo_df = build_transactions(effective_at, portfolio_name_fifo).query(conditions)
lifo_df = build_transactions(effective_at, portfolio_name_lifo).query(conditions)
high_df = build_transactions(effective_at, portfolio_name_high).query(conditions)
low_df = build_transactions(effective_at, portfolio_name_low).query(conditions)

display("Average tax lot", avg_df.loc[avg_df['type'] == "Sell"])
display("FIFO tax lot", fifo_df.loc[fifo_df['type'] == "Sell"])
display("LIFO tax lot", lifo_df.loc[lifo_df['type'] == "Sell"])
display("Highest tax lot", high_df.loc[high_df['type'] == "Sell"])
display("Lowest tax lot", low_df.loc[low_df['type'] == "Sell"])

'Average tax lot'

,txn_date,name,luid,sleeve,type,price,units,amount,sell_units,sell_price,sell_amount,gain/loss
23,2020-03-08,EQUITY_MSFT,LUID_00003IZ0,Sleeve 2,Sell,139.6,"80,000.0","11,164,800.0","80,000.0",142.6,"11,408,320.0","-243,520.0"


'FIFO tax lot'

,txn_date,name,luid,sleeve,type,price,units,amount,sell_units,sell_price,sell_amount,gain/loss
24,2020-03-08,EQUITY_MSFT,LUID_00003IZ0,Sleeve 2,Sell,139.6,"80,000.0","11,164,800.0","80,000.0",144.2,"11,538,000.0","-373,200.0"


'LIFO tax lot'

,txn_date,name,luid,sleeve,type,price,units,amount,sell_units,sell_price,sell_amount,gain/loss
24,2020-03-08,EQUITY_MSFT,LUID_00003IZ0,Sleeve 2,Sell,139.6,"80,000.0","11,164,800.0","70,000.0",140.2,"9,816,100.0","-46,900.0"


'Highest tax lot'

,txn_date,name,luid,sleeve,type,price,units,amount,sell_units,sell_price,sell_amount,gain/loss
24,2020-03-08,EQUITY_MSFT,LUID_00003IZ0,Sleeve 2,Sell,139.6,"80,000.0","11,164,800.0","80,000.0",144.2,"11,538,000.0","-373,200.0"


'Lowest tax lot'

,txn_date,name,luid,sleeve,type,price,units,amount,sell_units,sell_price,sell_amount,gain/loss
24,2020-03-08,EQUITY_MSFT,LUID_00003IZ0,Sleeve 2,Sell,139.6,"80,000.0","11,164,800.0","70,000.0",140.2,"9,816,100.0","-46,900.0"


Compare gain and loss for each tax lot

In [39]:
get_taxlot_facts([avg_df, fifo_df, lifo_df, high_df, low_df], ["Average", "FIFO", "LIFO", "Highest", "Lowest"])

You can also view the gain and loss by accessing the transactions screen and clicking on the +/- icon on the sell transactions. Use the links below.

In [40]:
generate_href(Demo.TRANSACTIONS, datetime(year=2020, month=3, day=11, tzinfo=pytz.UTC), portfolio_list)                                